In [1]:
from datasets import load_dataset

In [2]:
ds = load_dataset("squad")

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [4]:

tokenizer

BertTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [ ]:
ds["train"][0]

In [ ]:
inputs = tokenizer(
    ds["train"][2:6]["question"],
    ds["train"][2:6]["context"],
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)


In [ ]:
test_data = ds["train"][2:6]
answers = test_data["answers"]
start_positions = []
end_positions = []

In [ ]:
answers

In [ ]:
len(inputs["offset_mapping"]), len(inputs['input_ids'])

In [ ]:
ds["train"][2]

In [ ]:
len(tokenizer.tokenize(ds["train"][2]['context'])),tokenizer.tokenize(ds["train"][2]['context'])[:2]

In [ ]:
inputs.keys()

Input:

The tokenizer splits each question, context into max length and overlfows if more than max length. Here the same context can be split into multiple records

Start index of the character where answer begins

Goal: 

From the context find start index of the token corresponding to the answer in the context and the end index of the token corresponding to where the answer ends


Solution:

2. Take the length of the answer and find the last character index from start character index given for the answer
3. Offset mapping has start character and end characters for each token generated from input ids

first - 250, last - 270

(1,12),(13,14),..(247,249),(249,260),(261,275)

5. For each offset mapping token, check if the first character index of the token  == start character string then choose the index for that token within the input id
6. Similarly check if the last character index of the token >= last character index then return the index of the token within the input ID

Input ID is the combination of both question + context. Find the context before doing steps 3,4,5






In [5]:
def getstartandendtokens(inputs, answers):
        
    start_ans_index = []
    end_ans_index = []
    for index, offset in enumerate(inputs["offset_mapping"]):
        split = inputs["overflow_to_sample_mapping"][index]
        answer = answers[split]
        start_char_ans = answer['answer_start'][0]
        end_char_ans = start_char_ans+len(answer['text'][0])
       
        sequence_ids = inputs.sequence_ids(index)
    
        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1
        
        if offset[context_start][0] > start_char_ans or offset[context_end][1] < end_char_ans:
            start_ans_index.append(0)
            end_ans_index.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char_ans:
                idx += 1
            start_ans_index.append(idx - 1)
    
            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char_ans:
                idx -= 1
            end_ans_index.append(idx + 1)
    return start_ans_index,end_ans_index
    
            
    
    
    

In [ ]:
print(getstartandendtokens(inputs))

In [6]:
def preprocesstrainingdata(example):
    inputs = tokenizer(example["question"],
                      example["context"],
                      truncation = "only_second",
                      max_length = 384,
                      stride=128,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
                       padding="max_length",)
    answers = example["answers"]
    inputs["start_index"], inputs["end_index"] = getstartandendtokens(inputs,answers)
    return inputs
    


In [ ]:
ds["train"].column_names

In [7]:
tokenized_train_ds = ds["train"].map(preprocesstrainingdata, batched=True, remove_columns = ds["train"].column_names)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

In [ ]:
len(tokenized_train_ds) - len(ds["train"])

<b>Validation Set</b>

In [ ]:
ds["validation"]

In [8]:
def preprocessvalidationset(example):
    
    inputs = tokenizer(example["question"],
                            example["context"],truncation ="only_second",
                  max_length = 384,
                      stride=128,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
                       padding="max_length",)
    validation_sample = example["id"]
    validation_id = []
    context_id = []
    for item in range(len(inputs['input_ids'])):
        offset_id = inputs['overflow_to_sample_mapping'][item]
        validation_id.append(validation_sample[offset_id])
        offset = inputs['offset_mapping'][item]
        sequence = inputs.sequence_ids(item)
        inputs['offset_mapping'][item] = [value if sequence[i] == 1 else None for  i, value in enumerate(offset)]
    inputs["validation_id"] = validation_id
    return inputs


In [9]:
tokenized_ds_validation = ds["validation"].map(preprocessvalidationset, batched = True, remove_columns = ds["validation"].column_names)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
tokenized_ds_validation

In [10]:
small_eval_set = ds["validation"].select(range(100))
trained_checkpoint = "distilbert-base-cased-distilled-squad"

tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
eval_set = small_eval_set.map(
    preprocessvalidationset,
    batched=True,
    remove_columns=ds["validation"].column_names,
)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
eval_set

In [11]:
import torch
from transformers import AutoModelForQuestionAnswering

eval_set_for_model = eval_set.remove_columns(["validation_id", "offset_mapping","overflow_to_sample_mapping"])
eval_set_for_model.set_format("torch")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch = {k: eval_set_for_model[k].to(device) for k in eval_set_for_model.column_names}


In [12]:
trained_model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint).to(
    device
)



2025-09-24 19:33:40.068248: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758742420.290310      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758742420.357719      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

In [13]:
trained_model

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
     

In [14]:
with torch.no_grad():
    outputs = trained_model(**batch)
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

In [15]:
import collections

example_to_features = collections.defaultdict(list)
for idx, feature in enumerate(eval_set):
    example_to_features[feature["validation_id"]].append(idx)

In [16]:
example_to_features['56be4db0acb8001400a502ec']

[0]

In [17]:
import numpy as np
n_best = 20
max_answer_length =30
predicted_answers = []
for item in small_eval_set:
    id, context, answers = item['id'], item['context'], item['answers']
    answers = []
    for tok_index in example_to_features[id]:
        offset = eval_set["offset_mapping"][tok_index]
        start_logit = start_logits[tok_index]
        end_logit= end_logits[tok_index]
        start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
        end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
        #print(start_indexes, end_indexes)
        for start in start_indexes:
            for end in end_indexes:
                if offset[start] == None or offset[end]  == None:
                    continue
                if end < start or end - start + 1 > max_answer_length:
                    continue
                answers.append({
                    "text":context[offset[start][0]:offset[end][1]],
                "score":start_logit[start]+end_logit[end]}
                
                )
    best_answer = max(answers, key = lambda x: x["score"])
    predicted_answers.append({"id": id, "prediction_text": best_answer["text"]})

In [39]:
predicted_answers[:10]

[{'id': '56be4db0acb8001400a502ec', 'prediction_text': 'Denver Broncos'},
 {'id': '56be4db0acb8001400a502ed', 'prediction_text': 'Carolina Panthers'},
 {'id': '56be4db0acb8001400a502ee',
  'prediction_text': "Levi's Stadium in the San Francisco Bay Area at Santa Clara, California"},
 {'id': '56be4db0acb8001400a502ef', 'prediction_text': 'Carolina Panthers'},
 {'id': '56be4db0acb8001400a502f0', 'prediction_text': 'gold'},
 {'id': '56be8e613aeaaa14008c90d1', 'prediction_text': 'golden anniversary'},
 {'id': '56be8e613aeaaa14008c90d2', 'prediction_text': 'February 7, 2016'},
 {'id': '56be8e613aeaaa14008c90d3',
  'prediction_text': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference'},
 {'id': '56bea9923aeaaa14008c91b9', 'prediction_text': 'golden anniversary'},
 {'id': '56bea9923aeaaa14008c91ba',
  'prediction_text': 'American Football Conference'}]

In [31]:
theoretical_answers = [
    {"id": ex["id"], "answers": ex["answers"]} for ex in small_eval_set
]

In [35]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvid

In [36]:
import evaluate

metric = evaluate.load("squad")

In [37]:
metric

EvaluationModule(name: "squad", module_type: "metric", features: {'predictions': {'id': Value(dtype='string', id=None), 'prediction_text': Value(dtype='string', id=None)}, 'references': {'id': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}}, usage: """
Computes SQuAD scores (F1 and EM).
Args:
    predictions: List of question-answers dictionaries with the following key-values:
        - 'id': id of the question-answer pair as given in the references (see below)
        - 'prediction_text': the text of the answer
    references: List of question-answers dictionaries with the following key-values:
        - 'id': id of the question-answer pair (see above),
        - 'answers': a Dict in the SQuAD dataset format
            {
                'text': list of possible texts for the answer, as a list of strings
                'answer_start': list of start positions for 

In [40]:
results = metric.compute(predictions = predicted_answers, references =theoretical_answers )

In [41]:
results

{'exact_match': 83.0, 'f1': 88.25000000000004}